# Transforming and Cleaning ICD11 Data

In [22]:
import json

clean_data = dict()
foundation_calls = json.load(open('icd11_foundation.json','r'))
for call in foundation_calls:
    if 'narrowerTerm' in call.keys():
        display(call['narrowerTerm'])
        break
for call in foundation_calls:
    datum = dict()
    datum['title'] = call['title']['@value']
    datum['browser_url'] = call['browserUrl']
    datum['id'] = call['@id'].split('/')[-1]
    if 'parent' not in call.keys():
        call['parent'] = []
    if 'definition'  not in call.keys():
        call['definition'] = {"@value":''}
    if 'exclusion' not in call.keys():
        call['exclusion'] = []
    if 'inclusion' not in call.keys():
        call['inclusion'] = []
    if 'synoynm' not in call.keys():
        call['synoynm'] = []
    if 'narrowerTerm' not in call.keys():
        call['narrowerTerm'] = []
    parent_nodes = [parent_node.split('/')[-1] for parent_node in call['parent']]
    child_nodes = [child_node.split('/')[-1] for child_node in call['child']]
    exclusions = list()
    inclusions = list()
    synonyms = list()
    terms = list()
    for ex in call['exclusion']:
        if 'foundationReference' not in ex.keys():
            print('found exclusions')
            exclusions.append({'label':ex['label']['@value']})
        else:
            exclusions.append({'label':ex['label']['@value'], 'id':ex['foundationReference'].split('/')[-1]})
    for inc in call['inclusion']:
        if 'foundationReference' not in inc.keys():
            inclusions.append({'label':inc['label']['@value']})
        else:
            inclusions.append({'label':inc['label']['@value'], 'id':inc['foundationReference'].split('/')[-1]})
    for syn in call['synoynm']:
        if 'foundationReference' not in syn.keys():
            synonyms.append({'label':syn['label']['@value']})
        else:
            print('found synonym')
            synonyms.append({'label':syn['label']['@value'], 'id':syn['foundationReference'].split('/')[-1]})
    for term in call['narrowerTerm']:
        if 'foundationReference' not in term.keys():
            terms.append(term['label']['@value'])
        else:
            print('actually happened')
            terms.append({'label':term['label']['@value'], 'id':term['foundationReference'].split('/')[-1]})
    datum['parent_nodes'] = parent_nodes
    datum['parent_nodes'] = parent_nodes
    datum['child_nodes'] = child_nodes
    datum['definition'] = call['definition']['@value']
    datum['exclusions'] = exclusions
    datum['inclusions'] = inclusions
    datum['synonyms'] = synonyms
    datum['narrower_terms'] = terms
    clean_data[datum['id']] = datum
#change the key for the root node to 'root'
clean_data['root'] = clean_data['entity']
clean_data.pop('entity')
json.dump(clean_data,open('clean_icd11_foundation_data.json','w'))

[{'label': {'@language': 'en',
   '@value': 'injury or harm in therapeutic use of caffeine'}}]

found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions
found exclusions


In [12]:
import json
count = 0
foundation_calls = json.load(open('icd11_foundation.json','r'))
for datum in clean_data:
    if datum['exclusions']:
        count += 1
print(f'Number of codes with exclusions: {count}')

Number of codes with exclusions: 2875


In [32]:
import json
from collections import deque
clean_data =  json.load(open('clean_icd11_foundation_data.json','r'))
root = clean_data['root']
child_nodes = deque()
initial_nodes = root['child_nodes']
for node in initial_nodes:
    child_nodes.append(node)
    clean_data[node]['siblings'] = set(initial_nodes)
while child_nodes:
    current_id = child_nodes.popleft()
    current_node = clean_data[current_id]
    current_children = current_node['child_nodes']
    for child_id in current_children:
        if 'siblings' not in clean_data[child_id].keys():
            clean_data[child_id]['siblings'] = set(current_children)
        else:
            for new_child_id in current_children:
                clean_data[child_id]['siblings'].add(new_child_id)
        child_nodes.append(child_id)
json.dump(clean_data,open('clean_icd11_foundation_data_v2.json','r'))

pretty printing